# 2.01 - Modelling COB Peaks Timeseries
Using the peaks to assess the distribution of the data is a helpful approach to understand the distribution of meal intake over time. It has made it easier to assess the correctness of the data mapping to a daily pattern, especially given the issues with datetimes not aligning to the timezones that they are in. We'll now use the peaks to identify the COB values we are interested in modelling. The aim is to be able to assess what a standard day looks like and whether it is possible to idenfity where days are not standard, which may be due to errors in the data or due to the individual having a different pattern of meal intake. The peaks will be used to identify the COB values that are relevant for modelling, and then the timeseries will be used to assess the distribution and amplitude of those values over time. We will use the 15-minute resampled data here and focus on one of the candidates with the most defined distributions that shows a 3-meal intake clearly.

In [16]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from loguru import logger
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import CrostonClassic, CrostonSBA, CrostonOptimized, TSB

from src.cob_analysis import Cob
from src.data_processing.read import read_profile_offsets_csv
from src.configurations import Configuration
from src.time_series_analysis import run_adf, p_q_result, ts_dist_plot, ts_plot, split_ts, ts_plot_cfs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\.env\master_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logger.remove()

candidates = [13029224, 21946407, 27700103, 32407882, 41131654, 42360672, 67208817, 74175219, 79526193, 86025410, 95851255, 96254963, 96805916, 97417885]
individual = 41131654
args = {'height': 15, 'distance': 5, 'suppress': False}
config = Configuration()

profile_offsets = read_profile_offsets_csv(config)

cob = Cob()
cob.read_interim_data(file_name='15min_iob_cob_bg', sampling_rate=15)
df_all = cob.process_one_tz_individuals(profile_offsets, args)

Number of records: 786757
Number of people: 133
Systems used: 	['OpenAPS']
Categories (1, object): ['OpenAPS']
From 120 IDs requested processing, ignored 22 individuals not found in dataset, leaving 98 processed records.
The following stats are based on parameters h=15 and d=5:
	Number of records: 2637045
	Number of days with peaks: 7923
	Number of peaks: 19539


The data has a 'cob max' column that we need to transform such that it only holds the values that are relevant for modelling. The peaks will be used to identify the COB values that are used for features. That removes any noise from other values. Note, the imputed values are not used and would be irrelevant anyway, given that we are focussing purely on the values that are peaks only. These would alway be original values.

In [3]:
df = df_all.loc[individual].copy()
df.index.freq = str(cob.sampling_rate) + 'min'  # Avoid FutureWarning

In [4]:
# Running a quick ADF test to check if the data is stationary
run_adf(df['cob max'])

ADF Test Results
Null Hypothesis: The series has a unit root (non-stationary)
ADF-Statistic: -26.97359102321359
P-Value: 0.0
Number of lags: 53
Number of observations: 36363
Critical Values: {'1%': np.float64(-3.9590189861090983), '5%': np.float64(-3.4106107449475744), '10%': np.float64(-3.1271211082131924)}
Note: If P-Value is smaller than 0.05, we reject the null hypothesis and the series is stationary.
A more negative test statistic indicates stronger evidence against the null hypothesis.


This shows that the data is showing stationarity, despite the fact that we can see the clustering of peaks on a overimposed days. It is unsurprising considering: a) the data is not consistently entered by the individual, and b) we know the data is sparse, which combined will impact immediate lags. Therefore, we need to extend our approach to appreciate the seasonality, and are able to do so with  using the SARIMA model.

SARIMA is a seasonal extension of ARIMA, which is a popular time series forecasting method. It is particularly useful for data that exhibits seasonality, as it incorporates both non-seasonal and seasonal components. The SARIMA model can be expressed as:
$$
SARIMA(p, d, q)(P, D, Q)_s
$$
where:
- p: order of the non-seasonal autoregressive part
- d: degree of differencing for the non-seasonal part
-  q: order of the non-seasonal moving average part
- P: order of the seasonal autoregressive part
- D: degree of differencing for the seasonal part
- Q: order of the seasonal moving average part
- s: length of the seasonal cycle (e.g., 96 for 15-minute intervals over a day)
- The SARIMA model can be used to capture both the short-term and long-term patterns in the data, making it suitable for forecasting and understanding the underlying trends.

The model is defined by three main parameters: p, d, q (non-seasonal) and P, D, Q (seasonal), along with the seasonal period s. This seasonal period would be a day, considering we expect a person's eating pattern to be diurnal. The parameters would need to respond to the sampling rate. That used for example is 15 minutes, and the period would be 96 (24 hours * 60 minutes / 15 minutes).

However, with careful consideration, for this time series we don't witness a decay in the data points we are using as our random variable, therefore we should look to another method for this use. As a baseline, we'll look at Croston's Method (or SBA). `StatsForecast` offers a library of Croston-based (and similar) methods. The classic Croston's formulation is:
- $Y_t$ : Carbohydrate intake at time $t$.
- $Z_j$ : The $j^{th}$ non-zero carbohydrate intake value.
- $X_j$ : The $j^{th}$ inter-arrival time (number of periods between non-zero values).

Simple Exponential Smoothing (SES) for $Z_j$ and $X_j$:
$$s_j = \alpha \cdot Z_j + (1 - \alpha) \cdot s_{j-1}$$
where $\alpha$ is the smoothing parameter (0 < $\alpha$ < 1).
$$a_j = \beta \cdot X_j + (1 - \beta) \cdot a_{j-1}$$

The forecast for the next non-zero value is given by:
$$\hat{Y}_{t+h} = \frac{s_N}{a_N}$$
where $h$ is the forecast horizon and $N$ is the total number of non-zero intake events that have occurred in historical data up to the current time $t$. $s_N$ is the latest smoothed demand size, which is the exponentially smoothed average of all N non-zero demand values encountered in the historical data. $a_N$ is the latest smoothed interval, which is the exponentially smoothed average of all $N$ intervals between these non-zero demand occurrences.

Lets implement and see what the results look like. We will use the `StatsForecast` library to implement Croston's Method, and then plot the results to see how well it captures the peaks in the data.


In [41]:
# StatsForecast expects a specific DataFrame format or use the parameters. Here, we're aligned the column names to match the expected format: 'unique_id', 'ds' (datetime), 'y' (values)
df_croston = df.copy()
df_croston['y'] = np.where(df_croston['peak'] == 1, df_croston['cob max'], 0)
df_croston = (df_croston.
              reset_index().reset_index().
              rename(columns={'index': 'unique_id', 'datetime': 'ds'}).
              drop(columns=['cob max', 'day', 'time', 'cob interpolate', 'peak', 'offset']))

print("Sample Data Head (StatsForecast format):")
print(df_croston.head(10))
print("\nNumber of non-zero observations:", (df_croston['y'] > 0).sum())
print("Total observations:", len(df_croston))
print("Sparsity (proportion of zeros):", np.mean(df_croston['y'] == 0))

Sample Data Head (StatsForecast format):
   unique_id                  ds    y
0          0 2019-09-01 01:15:00  0.0
1          1 2019-09-01 01:30:00  0.0
2          2 2019-09-01 01:45:00  0.0
3          3 2019-09-01 02:00:00  0.0
4          4 2019-09-01 02:15:00  0.0
5          5 2019-09-01 02:30:00  0.0
6          6 2019-09-01 02:45:00  0.0
7          7 2019-09-01 03:00:00  0.0
8          8 2019-09-01 03:15:00  0.0
9          9 2019-09-01 03:30:00  0.0

Number of non-zero observations: 1269
Total observations: 36417
Sparsity (proportion of zeros): 0.9651536370376472


The result shows our data to be very sparse, as we would expect. Almost 97% zero-values. This can cause issues with the Croston's Method, especially if the smoothing parameters are not set correctly or if the data is too sparse, leading to forecasts that are too low or even zero due to numerical precision issues.

Some notes on the implementation using StatsForecast:
- For Croston's, you generally don't set alpha/beta directly in `CrostonClassic`/`SBA` as they use default values or optimized values. `CrostonOptimized` allows you to tune alpha and beta, or it can optimize them.
- The `TSB` model requires you to specify smoothing parameters (`alpha_d` for demand and `alpha_p` for period), which can be tuned based on your data characteristics.

In [43]:
def run_croston_models(df, sampling_rate=15):
    """
    Run Croston's models on the provided DataFrame.
    Returns a DataFrame with forecasts for each model.
    """
    intervals_per_day = 24 * 60 // sampling_rate

    models = [
        CrostonClassic(),  # Classic Croston's with default alpha=0.1 for both
        CrostonSBA(),      # Croston's with Syntetos-Boylan Approximation (bias-corrected)
        CrostonOptimized(),# Optimized Croston's (finds best alpha/beta within a range)
        TSB(alpha_d=0.2, alpha_p=0.2) # TSB model, requires smoothing parameters
    ]

    sf = StatsForecast(
        models=models,
        freq=str(sampling_rate)+'min',
        n_jobs=-1 # Use all available CPU cores for parallel processing
    )

    sf.fit(df=df)

    return sf.predict(h=intervals_per_day)  # Predict for the next 96 intervals (1 day), returns df

def print_croston_forecasts(df, sampling_rate=15):
    """
    Print the forecasts from Croston's models.
    """
    intervals_per_day = 24 * 60 // sampling_rate

    print(f"\nCrostonClassic Forecast (per 15-min interval): {df['CrostonClassic'].iloc[0]:.4f}")
    print(f"CrostonSBA Forecast (per 15-min interval): {df['CrostonSBA'].iloc[0]:.4f}")
    print(f"CrostonOptimized Forecast (per 15-min interval): {df['CrostonOptimized'].iloc[0]:.4f}")
    print(f"TSB Forecast (per 15-min interval): {df['TSB'].iloc[0]:.4f}")

    # Calculate average daily intake for each model
    for model_name in df.columns[2:]: # Skip 'unique_id' and 'ds'
        daily_avg = df[model_name].iloc[0] * intervals_per_day
        print(f"Average Daily Forecast ({model_name}): {daily_avg:.2f} units")

forecast_df = run_croston_models(df_croston)
print_croston_forecasts(forecast_df)


# sampling_rate = 15
# intervals_per_day = 24 * 60 // sampling_rate  # Number of intervals in a day for 15-min data
#
# models = [
#     CrostonClassic(),  # Classic Croston's with default alpha=0.1 for both
#     CrostonSBA(),      # Croston's with Syntetos-Boylan Approximation (bias-corrected)
#     CrostonOptimized(),# Optimized Croston's (finds best alpha/beta within a range)
#     TSB(alpha_d=0.2, alpha_p=0.2) # TSB model, requires smoothing parameters
# ]
#
# sf = StatsForecast(
#     models=models,
#     freq=str(sampling_rate)+'min',
#     n_jobs=-1 # Use all available CPU cores for parallel processing
# )
#
# sf.fit(df=df_croston)
#
# # Here we predict for the next 96 intervals (1 day) using the fitted models.
# # The idea being that this produces a forecast for a standard day.
# forecast_df = sf.predict(h=intervals_per_day)
# print("\nForecasts from StatsForecast models:")
# print(forecast_df.head()) # Note that forecasts are constant for intermittent models
#
# print(f"\nCrostonClassic Forecast (per 15-min interval): {forecast_df['CrostonClassic'].iloc[0]:.4f}")
# print(f"CrostonSBA Forecast (per 15-min interval): {forecast_df['CrostonSBA'].iloc[0]:.4f}")
# print(f"CrostonOptimized Forecast (per 15-min interval): {forecast_df['CrostonOptimized'].iloc[0]:.4f}")
# print(f"TSB Forecast (per 15-min interval): {forecast_df['TSB'].iloc[0]:.4f}")
# # Calculate average daily intake for each model
# for model_name in forecast_df.columns[2:]: # Skip 'unique_id' and 'ds'
#     daily_avg = forecast_df[model_name].iloc[0] * intervals_per_day
#     print(f"Average Daily Forecast ({model_name}): {daily_avg:.2f} units")


CrostonClassic Forecast (per 15-min interval): 0.0000
CrostonSBA Forecast (per 15-min interval): 0.0000
CrostonOptimized Forecast (per 15-min interval): 0.0000
TSB Forecast (per 15-min interval): 0.0000
Average Daily Forecast (CrostonClassic): 0.00 units
Average Daily Forecast (CrostonSBA): 0.00 units
Average Daily Forecast (CrostonOptimized): 0.00 units
Average Daily Forecast (TSB): 0.00 units


The zero forecasts may be for a number of reasons:
- The data is too sparse, leading to numerical precision issues.
- The smoothing parameters are not set correctly, leading to forecasts that are too low or even zero.
- The model is not able to capture the intermittent nature of the data, leading to forecasts that are not representative of the underlying demand.
- The model is not able to capture the seasonality of the data, leading to forecasts that are not representative of the underlying demand.
- Initialisation of $s_0$ and $a_0$ in Croston's Method can lead to zero forecasts if the initial values are not set correctly.

Following previous visual checks, we have noted with this individual that they have a period of days in which they have no peaks, and therefore no data to model. This is likely to be the cause of the zero forecasts. We can check this by looking at the number of non-zero observations in the forecast data. We've created a function to remove days with zero intake from the data, so lets apply this and see if that changes the forecasts.

In [44]:
from src.cob_analysis import remove_zero_or_null_days
print(f'Before removing zero intake days: {len(df.groupby("day").size())}')
df_croston = df_croston.set_index('ds')  # Ensure 'ds' is the index for time series operations
df_croston = remove_zero_or_null_days(df=df_croston, value_col='y')
print(f'After removing zero intake days: {len(df_croston.groupby(df_croston.index.date).size())}')
df_croston = df_croston.reset_index()  # Reset index to keep 'ds' as a column
print(df_croston.info())

Before removing zero intake days: 380
After removing zero intake days: 365
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34977 entries, 0 to 34976
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         34977 non-null  datetime64[ns]
 1   unique_id  34977 non-null  int64         
 2   y          34977 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 819.9 KB
None


In [45]:
forecast_df = run_croston_models(df_croston)
print_croston_forecasts(forecast_df)


CrostonClassic Forecast (per 15-min interval): 0.0000
CrostonSBA Forecast (per 15-min interval): 0.0000
CrostonOptimized Forecast (per 15-min interval): 0.0000
TSB Forecast (per 15-min interval): 0.0000
Average Daily Forecast (CrostonClassic): 0.00 units
Average Daily Forecast (CrostonSBA): 0.00 units
Average Daily Forecast (CrostonOptimized): 0.00 units
Average Daily Forecast (TSB): 0.00 units


In [ ]:
# --- 5. Visualization (Optional) ---
# Merge historical data with forecasts for plotting
# The forecast for intermittent models is a constant line
plot_df = pd.concat([df_croston, forecast_df.drop(columns='unique_id')], axis=0)
plt.figure(figsize=(15, 7))
plt.plot(plot_df['ds'], plot_df['y'], label='Historical Carb Intake', alpha=0.7)
plt.plot(forecast_df['ds'], forecast_df['CrostonSBA'], label='CrostonSBA Forecast', color='red', linestyle='--')
plt.plot(forecast_df['ds'], forecast_df['TSB'], label='TSB Forecast', color='green', linestyle=':')
plt.title('Carbohydrate Intake: Historical Data vs. Intermittent Demand Forecasts')
plt.xlabel('Time')
plt.ylabel('Carbohydrate Intake (units)')
plt.legend()
plt.grid(True)
plt.show()
# Zoomed-in plot to see the intermittent nature better
plt.figure(figsize=(15, 7))
plt.plot(plot_df['ds'].tail(intervals_per_day * 5), plot_df['y'].tail(intervals_per_day * 5), label='Historical Carb Intake', alpha=0.7)
plt.plot(forecast_df['ds'], forecast_df['CrostonSBA'], label='CrostonSBA Forecast', color='red', linestyle='--')
plt.plot(forecast_df['ds'], forecast_df['TSB'], label='TSB Forecast', color='green', linestyle=':')
plt.title('Carbohydrate Intake (Last 5 Days): Historical Data vs. Intermittent Demand Forecasts')
plt.xlabel('Time')
plt.ylabel('Carbohydrate Intake (units)')
plt.legend()
plt.grid(True)
plt.show()

In [37]:
forecast_df.describe()

,unique_id,ds,CrostonClassic,CrostonSBA,CrostonOptimized,TSB
count,3.496032e+06,3496032,3.496032e+06,3.496032e+06,3.496032e+06,3.496032e+06
mean,1.820800e+04,2020-03-09 05:22:30.000000512,1.510613e+00,1.435083e+00,1.510613e+00,1.510613e+00
min,0.000000e+00,2019-09-01 01:30:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.104000e+03,2019-12-05 09:15:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.820800e+04,2020-03-09 05:22:30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.731200e+04,2020-06-12 01:30:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.641600e+04,2020-09-15 09:15:00,1.200000e+02,1.140000e+02,1.200000e+02,1.200000e+02
std,1.051268e+04,NaN,8.908414e+00,8.462993e+00,8.908414e+00,8.908414e+00
